In [1]:
%matplotlib inline
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
import scipy
import seaborn as sns
import pylab as plt

Populating the interactive namespace from numpy and matplotlib


In [ ]:
### 2018 Orbis snapshot

## ALL positions
# In BE (Belgium), total number of ALL positions: 3,242,838
# positionsALL with known appDate: 0.697
# positionsALL with known resDate 0.4949


## IMPortant positions
# In BE (Belgium), total number of IMP positions: 1,263,539
# positionsIMP with known appDate: 0.5072
# positionsIMP with known resDate: 0.19089


#SELECT country, COUNT(*) as appointments, 
#SUM(CASE WHEN appDate IS NOT NULL THEN 1 ELSE 0 END)/COUNT(*) as appDateKnown, 
#SUM(CASE WHEN resDate IS NOT NULL THEN 1 ELSE 0 END)/COUNT(*) as resDateKnown
#FROM positionsIMP
#WHERE country = 'BE'

In [ ]:
# Take ALL positions
SELECT
p.BvD as BvD,
p.UCI as positionUCI,
p.appDate as appDate,
p.resDate as resDate,
p.fullName as fullName,
p.title as title,
p.type as type,
p.levelResponsability as responsability,
p.country as country
FROM positionsALL p
WHERE country = 'BE'
AND individual = 1 # directors == individuals

In [2]:
df0 = pd.read_csv('/home/diliara/Dropbox/global cores/BE/be_sql_all.csv', sep=',', low_memory=False, header=0)
df0.head()

,﻿BvD,positionUCI,appDate,resDate,fullName,title,type,responsability,country
0,AU107607670-B0001,P161600702,NaN,NaN,Ms Julie Sandrine Katia Didi De Nul,Branch Manager,BrOff,Branch manager,BE
1,AU107607670-B0001,P044147859,NaN,NaN,Mr Gery Maurice Cornelius Vandewalle,Branch Manager,BrOff,Branch manager,BE
2,AU110717336-B0002,P199588153,NaN,NaN,Mr Thomas Linnwood Beaudoin,Branch Manager,BrOff,Branch manager,BE
3,AU110717336-B0002,P039303546,NaN,NaN,Mr Jan Louis Cornelius Anthierens,Branch Manager,BrOff,Branch manager,BE
4,AU153253537-B0002,P039022638,NaN,NaN,Mr Michel Jacques Bernard Etienne Moser,Branch Manager,BrOff,Branch manager,BE


In [3]:
print(df0["appDate"].isnull().sum()/len(df0))
# 33% of apps are not known

0.327615416901


In [4]:
print(df0["resDate"].isnull().sum()/len(df0))
# 54% of res are not known

0.535932770138


In [5]:
# delete cases where appDate and resDate are both null
df1 = df0.dropna(subset=['appDate'])
df2 = df1.dropna(subset=['resDate'])

In [6]:
print(len(df2)/len(df0))
# when we have only cases with known app and res dates, we have only 0.43 from the main dataset

0.428562877127383


In [7]:
# calclulate unique BvD and UCI groups
df2.rename(columns={'﻿BvD': 'BvD'}, inplace=True)
bvd_groups = df2.groupby(['BvD'])
len(bvd_groups)

/home/diliara/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


292563

In [8]:
uci_groups = df2.groupby(['positionUCI'])
len(uci_groups)
# There are 292563 unique firms and 547305 unique individuals

547305

## Cleaning

In [9]:
# transform time
df2['appDate2'] = pd.to_datetime(df2['appDate'], format='%Y%m%d')
df2['appDate2'].min()

/home/diliara/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Timestamp('1947-02-28 00:00:00')

In [11]:
df2['resDate2'] = pd.to_datetime(df2['resDate'], format='%Y%m%d')
df2['resDate2'].min()

/home/diliara/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Timestamp('1953-02-27 00:00:00')

In [12]:
# appDate
# delete cases > 1 Jan 2017
df3 = df2[df2["appDate"] < 20171231]

# delete cases < 1 Jan 2000
df4 = df3[df3["appDate"] > 19991231]

# resDate
df5 = df4[df4["resDate"] < 20171231]
df6 = df5[df5["resDate"] > 19991231]

In [13]:
# delete cases when BvD and UCI and date are the same (e.g. appointment to the the same firm but with different positions)
df7 = df6.drop_duplicates(subset=['BvD', 'positionUCI','appDate'], keep='first')

In [14]:
len(df7)

853608

## Exploring data

In [16]:
#For each BvD, assign the number of distinct directors it had over 2000-2017
df7['uniqueDirs'] = df7.groupby(['BvD'])["positionUCI"].transform(lambda x: len(x.unique()))
df7['uniqueDirs'].value_counts()

/home/diliara/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


3      180274
2      145767
4      120526
1      102370
5       69527
6       44577
7       29467
8       23037
9       17867
10      15228
11      12655
12      11545
13       9127
14       7399
15       6561
16       5194
17       4967
18       3914
19       3605
20       3351
21       3099
22       2704
23       2512
24       2143
25       1642
28       1540
27       1503
26       1357
29       1203
36       1141
        ...  
65        281
40        263
53        256
237       246
98        242
81        239
56        212
94        211
54        210
41        202
76        200
115       190
87        190
69        177
160       172
66        171
134       151
72        146
79        146
138       139
60        134
99        123
89        110
64        101
83         92
82         91
73         81
71         76
62         66
55         60
Name: uniqueDirs, dtype: int64

In [19]:
# Number of positions per individual
df7['noPositions'] = df7.groupby(["positionUCI"])['BvD'].transform(lambda x: len(x.unique()))
# it shows the number of positions a director had in different firms over 2000-2017
df7['noPositions'].value_counts()

/home/diliara/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


1      401374
2      157451
3       81762
4       48216
5       31751
6       22014
7       16834
8       12481
9       10425
10       8307
11       6991
12       5729
13       5501
14       3599
16       3594
15       3465
17       2785
18       2279
20       1879
19       1618
23       1606
21       1543
28       1293
25       1243
29       1219
24       1200
22       1128
26       1103
30       1004
34        831
        ...  
69        169
82        168
167       167
74        166
148       150
93        143
51        140
57        139
124       136
50        135
102       128
60        123
73        114
79        109
53        107
100       102
62        100
47         99
78         95
88         91
59         87
70         83
80         83
54         82
67         82
68         80
52         79
77         77
61         61
48         52
Name: noPositions, dtype: int64

In [20]:
df7['uniqueDirs'].max()

317

In [21]:
df7['noPositions'].max()

279

In [22]:
df7.loc[df7['uniqueDirs'] == 317]

,BvD,positionUCI,appDate,resDate,fullName,title,type,responsability,country,appDate2,resDate2,uniqueDirs,noPositions
65362,BE0403201185,P231990611,20040712.0,20110817.0,Mr Frank Lierman,Agent,SenMan,Representative,BE,2004-07-12,2011-08-17,317,1
65363,BE0403201185,P215526390,20040712.0,20110817.0,Mr Patricia Van Cappellen,Agent,SenMan,Representative,BE,2004-07-12,2011-08-17,317,1
65364,BE0403201185,P215526317,20051220.0,20110817.0,Mr Willy Van Cauwelaert,Agent,SenMan,Representative,BE,2005-12-20,2011-08-17,317,1
65365,BE0403201185,P215526456,20040712.0,20110817.0,Mr Gerrit Van Daele,Agent,SenMan,Representative,BE,2004-07-12,2011-08-17,317,1
65366,BE0403201185,P123517292,20040712.0,20110817.0,Mr Guy Van Damme,Agent,SenMan,Representative,BE,2004-07-12,2011-08-17,317,1
65367,BE0403201185,P124122539,20040712.0,20110817.0,Mr Gert Carl Hendrik Van De Genachte,Agent,SenMan,Representative,BE,2004-07-12,2011-08-17,317,1
65368,BE0403201185,P215526490,20040712.0,20110817.0,Mr Pascal Van De Meirssche,Agent,SenMan,Representative,BE,2004-07-12,2011-08-17,317,1
65369,BE0403201185,P124088567,20040712.0,20110817.0,Ms Cecile Van De Moosdyk,Agent,SenMan,Representative,BE,2004-07-12,2011-08-17,317,1
65370,BE0403201185,P215526502,20040712.0,20110817.0,Mr Herman Van De Voorde,Agent,SenMan,Representative,BE,2004-07-12,2011-08-17,317,2
65371,BE0403201185,P215526318,20040712.0,20110817.0,Mr Albert Van Den Borre,Agent,SenMan,Representative,BE,2004-07-12,2011-08-17,317,1


In [23]:
# remove individuals with more than 100 positions
df8 = df7[df7['noPositions'] < 101]

# remove firms with more than 100 individuals
df9 = df8[df8['uniqueDirs'] < 101]

In [24]:
# Position types
df9.type.value_counts()

BoD            441151
SenMan         370233
BoD,SenMan      38703
Proxy             753
ExeB              136
BoD,OthDep         62
SupB                3
AudC                1
ExeB,SenMan         1
ExeC,OthBC          1
BoD,OthBC           1
Name: type, dtype: int64

In [25]:
df9.title.value_counts()

Director                                                                407721
Business Manager                                                        171951
Managing Director                                                       136892
Chairman  Managing Director                                              37427
Chairman of the Board of Directors                                       27609
Manager                                                                  25803
Liquidator                                                               23480
Vice Chairman of the Board of Directors                                   4586
Company Secretary                                                         2915
Legal Representative                                                      2387
Senior Manager                                                            2278
Agent                                                                     1590
Vice-Chairman of the Board of Directors             

In [26]:
df9['duration'] = df9['resDate2'] - df9['appDate2']
df9.duration.describe()

/home/diliara/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


count                       851045
mean     1640 days 08:05:06.427979
std      1116 days 21:20:21.684436
min                0 days 00:00:00
25%              731 days 00:00:00
50%             1533 days 00:00:00
75%             2208 days 00:00:00
max             6498 days 00:00:00
Name: duration, dtype: object

In [27]:
# there are people with 0 days of duration
# delete cases if duration > 90 days (~3 months)
df10 = df9[df9['duration'] > '90 days 00:00:00']

In [28]:
len(df10)

833520

In [29]:
# calclulate unique BvD and UCI groups
bvd_groups = df10.groupby(['BvD'])
len(bvd_groups)

247462

In [30]:
uci_groups = df10.groupby(['positionUCI'])
len(uci_groups)

431746

In [32]:
len(df10)

833520

In [ ]:
# 247,462 firms, 431,746 individuals
# 833,520 board positions

## Components

In [33]:
# Creating a graph
import networkx as nx
from networkx.algorithms import bipartite

dk_graph = df10[['BvD','positionUCI']]
BVD = dk_graph['BvD']
UCI = dk_graph['positionUCI']
list_graph = dk_graph.values.tolist()

gr01 = nx.Graph()
gr01.add_nodes_from(BVD, bipartite=0)
gr01.add_nodes_from(UCI, bipartite=1)
gr01.add_edges_from(list_graph)
nx.is_bipartite(gr01)

True

In [34]:
## Number of connected components in a graph
nx.number_connected_components(gr01)

123047

In [35]:
# Size of the first 3 components
giant1 = sorted(nx.connected_component_subgraphs(gr01), key=len, reverse=True)[0]
giant2 = sorted(nx.connected_component_subgraphs(gr01), key=len, reverse=True)[1]
giant3 = sorted(nx.connected_component_subgraphs(gr01), key=len, reverse=True)[2]

len(giant1),len(giant2), len(giant3)

# 234,716 nodes in the first component

(234716, 144, 122)

In [36]:
# sava file for R
df10.to_csv('be_data_r.csv')